# Titanic Competition

This public kaggle competition, which is hosted <a href=https://www.kaggle.com/c/titanic>here</a>, aims at predicting the survival of passengers from Titanic. A dataset containing 891 observations and 10 features is provided for training.

* [1) Reading the data](#1-reading-the-data)
* [2) Preparing the data](#2-preparing-the-data)
* [3) Choosing the model](#3-choosing-the-model)
* [4) Tuning the model](#4-tuning-the-model)
* [5) Predicting classes for test data](#5-predicting-classes-for-test-data)

## 1) Reading the data <a class="anchor" id="1-reading-the-data"></a>

In [1]:
import numpy as np
import pandas as pd
import warnings

from sklearn.preprocessing import StandardScaler

In [2]:
raw_data = pd.read_csv('data/train.csv')
len(raw_data)

891

In [3]:
raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


One can notice that the column *Age* is probably going to cause some issues. Let us check the missing values.

In [4]:
raw_data.isnull().sum(axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

There are a lot of ages missing (around 20%) which means that we will have to deal with this issue properly.

There are also a lot of observations with no *Cabin* features (around 75%). So for the sake of simplicity here, we will first not consider the *Cabin* feature.

Also for the sake of simplicity, we will at first not use here *Ticket* (which would probably need some *regex* work, and *Name*.

## 2) Preparing the data <a class="anchor" id="2-preparing-the-data"></a>

In [5]:
# Drop unused features
raw_data=raw_data.drop(['Name','Ticket','Cabin'],axis=1).set_index('PassengerId')

# Fill missing Embarked with a new letter, "X"
raw_data['Embarked']=raw_data['Embarked'].fillna('X')

In [6]:
# Prepare X_train and y_train, on which the training will be done
X_train = raw_data.drop('Survived',axis=1)
y_train = raw_data['Survived']

In [7]:
# Check X_train out
X_train.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22.0,1,0,7.2500,S
2,1,female,38.0,1,0,71.2833,C
3,3,female,26.0,0,0,7.9250,S
4,1,female,35.0,1,0,53.1000,S
5,3,male,35.0,0,0,8.0500,S
6,3,male,NaN,0,0,8.4583,Q
7,1,male,54.0,0,0,51.8625,S
8,3,male,2.0,3,1,21.0750,S
9,3,female,27.0,0,2,11.1333,S


### 2.a) Dummifying categorical features

Embarked is categorical. Let's dummify it (*i.e.* create one boolean column per category value) in order to get numerical values out of it.

In [8]:
# Create dummy columns
aux = pd.get_dummies(X_train[['Embarked']])

# Add dummy columns to X_train
X_train = X_train.drop(['Embarked'],1).join(aux)

# Clean a bit
X_train = X_train.drop(['Embarked_X'],1)
del aux

In [9]:
# Let's check we do not have missing values anymore (apart from ages, which will be handled separately)
X_train.count(axis=0)

Pclass        891
Sex           891
Age           714
SibSp         891
Parch         891
Fare          891
Embarked_C    891
Embarked_Q    891
Embarked_S    891
dtype: int64

In [10]:
# Let's check out X_train now
X_train.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,male,22.0,1,0,7.2500,0.0,0.0,1.0
2,1,female,38.0,1,0,71.2833,1.0,0.0,0.0
3,3,female,26.0,0,0,7.9250,0.0,0.0,1.0
4,1,female,35.0,1,0,53.1000,0.0,0.0,1.0
5,3,male,35.0,0,0,8.0500,0.0,0.0,1.0


In [11]:
X_train.Pclass = X_train.Pclass-2
X_train.Sex = X_train.Sex.apply(lambda x: 1 if x=='male' else -1)

X_train.Embarked_C = 2*(X_train.Embarked_C-0.5)
X_train.Embarked_Q = 2*(X_train.Embarked_Q-0.5)
X_train.Embarked_S = 2*(X_train.Embarked_S-0.5)

X_train.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,1,1,22.0,1,0,7.2500,-1.0,-1.0,1.0
2,-1,-1,38.0,1,0,71.2833,1.0,-1.0,-1.0
3,1,-1,26.0,0,0,7.9250,-1.0,-1.0,1.0
4,-1,-1,35.0,1,0,53.1000,-1.0,-1.0,1.0
5,1,1,35.0,0,0,8.0500,-1.0,-1.0,1.0
6,1,1,NaN,0,0,8.4583,-1.0,1.0,-1.0
7,-1,1,54.0,0,0,51.8625,-1.0,-1.0,1.0
8,1,1,2.0,3,1,21.0750,-1.0,-1.0,1.0
9,1,-1,27.0,0,2,11.1333,-1.0,-1.0,1.0


Let's now create a function which will be doing all this for us. This is a common practice which is used, in order to have easily the exact same preparations done to the test dataset(s) afterwards.

In [12]:
def prepare_dataset(X):
    
    # Drop unused features
    X=X.drop(['Name','Ticket','Cabin'],axis=1).set_index('PassengerId')

    # Fill missing Embarked with a new letter, "X"
    X['Embarked']=X['Embarked'].fillna('X')
    
    # Create dummy columns
    aux = pd.get_dummies(X[['Embarked']])

    # Add dummy columns to X_train
    X = X.drop(['Embarked'],1).join(aux)

    # Clean a bit
    if 'Embarked_X' in X.columns:
        X = X.drop(['Embarked_X'],1)

    X.Pclass = X.Pclass-2
    X.Sex = X.Sex.apply(lambda x: 1 if x=='male' else -1)

    X.Embarked_C = 2*(X.Embarked_C-0.5)
    X.Embarked_Q = 2*(X.Embarked_Q-0.5)
    X.Embarked_S = 2*(X.Embarked_S-0.5)
    
    return X

### 2.b) Filling empty ages: why pipelines are important

For now, ages will be filled by the mean of the column Age. We will use a sklearn Imputer to do so. These are well-integrated in the sklearn workflow, just as well as sklearn scalers for instance, which enables to use cross-validation without doing the typical error of introducing "future information" while doing so.

Let me illustrate this with a simple example. I have a dataset, on which I want to perform cross-validation. If the imputer and scaler are applied to the whole dataset and then cross_val_score is used on the output, then it means the that model is trained (during cross-validation) on data depending on the test set, since scaler and imputer were applied to both train and test before the cross-validation's split.

The solution is to create a pipeline imputer-scaler-model and use this one as input to the cross_val_score function. This way the split train-test is done, the whole pipeline is fit on train (i.e. also the imputer and the scaler) and is then applied to test. No "future information" (i.e. information from the test set) is used to fit the imputer, scaler and model.

Plus, the imputer is applied to all columns. Thus, if there is one missing value in another column where there was no missing value in the training set, this will be handled automatically.

## 3) Choosing the model <a class="anchor" id="3-choosing-the-model"></a>

In [13]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline, make_union

In [14]:
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer

In [15]:
from sklearn.linear_model import LogisticRegression as RegLog
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [16]:
# Let's create a loop of models so we can iterate over them
Models = [RegLog, SVC, LinearSVC, NB, KNN, RandomForestClassifier, AdaBoostClassifier]

Let us now create an imputer and a scaler, for the purpose explained earlier. For now, we choose a simple imputing strategy: fill by the mean. This will be applied to each column separately.

In [17]:
imputer = Imputer(strategy='mean', axis=0)
scaler = StandardScaler()

Let us now create transformers which will extract the features on which we want to apply the scaler (called here "scalable data"), and the features on which we do not (called here "scalable data"). This will be the second step of the pipeline, after the imputer part.

Then we will create a union between {extract unscalable data} and {extract scalable data, then scale it}: they will be done separately and then stacked together.

Here, since the imputer outputs an array (rather than a DataFrame), we need to handle columns as indexes rather than lists of names.

In [18]:
ind_scalable = [2, 3, 4, 5]
X_train.columns[ind_scalable]

Index(['Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [19]:
ind_unscalable = [0, 1, 6, 7, 8]
X_train.columns[ind_unscalable]

Index(['Pclass', 'Sex', 'Embarked_C', 'Embarked_Q', 'Embarked_S'], dtype='object')

In [20]:
def get_scalable(array):
    return array[:,ind_scalable]
def get_unscalable(array):
    return array[:,ind_unscalable]

In [21]:
get_scalable_ft = FunctionTransformer(get_scalable, validate=False)
get_unscalable_ft = FunctionTransformer(get_unscalable, validate=False)

Both extraction transformers are now created. Let's prepare pipelines:

In [22]:
scaler_pipeline = make_pipeline(get_scalable_ft, scaler)
scaler_union = make_union(scaler_pipeline, get_unscalable_ft)

Now, as explained earlier, the idea is to loop over models and choose the one is performing best. The next step will be to tune the chosen model.

This is how our workflow is working:

* For every model:
    * split between train and test
    * For every 4-1 combination of train-test:
        * fit_transform imputer on train
        * fit_transform our scaler pipeline on train
        * fit the model on the transformed train
        * transform test
        * predict test and check accuracy

In [23]:
warnings.filterwarnings('ignore')
for Model in Models:
    
    model = Model()
    main_pipe = make_pipeline(imputer, scaler_union, model)
    
    scores = cross_val_score(main_pipe, X_train, y_train, cv=5, scoring='accuracy')
    print(np.mean(scores))

0.785686011036
0.827165172405
0.78788919163
0.772303227159
0.809281658508
0.823895031788
0.800305377676


The best model seems to be SVC. In addition to that, SVC has many parameter tuning options. Thus, we choose SVC as our model.

## 4) Tuning the model <a class="anchor" id="4-tuning-the-model"></a>

The model was chosen in the previous section. Let's now try to tune it.

In [24]:
from sklearn.grid_search import RandomizedSearchCV

In [25]:
# Redefining the model and the main pipeline (all the rest is kept from previous section)
model = SVC()
main_pipe = make_pipeline(imputer, scaler_union, model)

In [26]:
main_pipe.named_steps.keys()

dict_keys(['imputer', 'svc', 'featureunion'])

We will use RandomizedSearchCV to tune our model efficiently. The data consists of only 9 features and less than 10k observations, so it is going to be very fast. We will thus be able to require a high number of iterations, and thus have an efficient tuning of our parameters.

This is where we use the full potential of the sklearn workflow, and the fact that we prepared everything properly (with unions and pipelines).

In [27]:
from scipy import stats

**Note:** for some parameters, the best value was found to be the default one. Those are commented in the cell below.

In [28]:
param_grid = {'svc__C': stats.uniform(loc=0, scale=10),
              # 'svc__class_weight':[None, 'balanced'],
              # 'svc__kernel':['linear', 'poly', 'rbf', 'sigmoid'],
              'svc__decision_function_shape':[None, 'ovo', 'ovr'],
              'svc__shrinking': [True, False]
             }
rand = RandomizedSearchCV(main_pipe, param_grid, cv=5, scoring='accuracy', n_iter=100)

In [29]:
# We fit to the train sets
rand.fit(X_train, y_train);

In [30]:
print('Best score: ', rand.best_score_)
print('Best params: ', rand.best_params_)

Best score:  0.830527497194
Best params:  {'svc__C': 2.536859800097081, 'svc__decision_function_shape': 'ovo', 'svc__shrinking': False}


## 5) Predicting classes for test data <a class="anchor" id="5-predicting-classes-for-test-data"></a>

In [31]:
# Load test data
raw_data_test = pd.read_csv('data/test.csv')
len(raw_data_test)

418

In [32]:
# Apply all dataset preparations to test set
X_test = prepare_dataset(raw_data_test)
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
892,1,1,34.5,0,0,7.8292,-1.0,1.0,-1.0
893,1,-1,47.0,1,0,7.0000,-1.0,-1.0,1.0
894,0,1,62.0,0,0,9.6875,-1.0,1.0,-1.0
895,1,1,27.0,0,0,8.6625,-1.0,-1.0,1.0
896,1,-1,22.0,1,1,12.2875,-1.0,-1.0,1.0


RandomizedSearchCV automatically fits the best model on the whole dataset. So our *rand* object can be used directly to predict the class of every observation in the test set.

In [33]:
y_pred = rand.predict(X_test)

Let's store this all into a file, and submit it on the competition!

In [34]:
output = pd.DataFrame({'PassengerId':X_test.index, 'Survived':y_pred})

In [35]:
output.to_csv('data/prediction.csv', index=False, header=True)

78.9% accuracy is obtained. This is about 4% lower than the accuracy reached through cross-validation, which is a bit disappointing. This might be caused by several things:
* Heterogenity between train and test dataset
* Overfitting of the model on the data
* Bad approximation of NA values

To improve it, several things could be tried:
* Increase penalization to reduce over-fitting
* Use a model (KNN for example) to predict missing ages in a smarter way than just filling by the mean